In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import json
from pathlib import Path
from typing import Any
from sklearn.metrics import mean_absolute_error

In [2]:
def clean_data(pushback, etd, lamp, mfs):
    #Take out unwanted columns
    lamp = lamp.drop('wind_direction', axis=1)
    mfs = mfs[mfs['isdeparture'] == True]
    #Make new timestamps and forecast timestamps to prep for merges
    lamp['timestamp_plus_30'] = lamp['timestamp'] + pd.Timedelta(minutes=30)
    lamp['timestamp_plus_45'] = lamp['timestamp'] + pd.Timedelta(minutes=45)
    lamp['timestamp_plus_60'] = lamp['timestamp'] + pd.Timedelta(minutes=60)
    lamp['timestamp_plus_75'] = lamp['timestamp'] + pd.Timedelta(minutes=75)
    lamp['forecast_plus_15'] = lamp['forecast_timestamp'] + pd.Timedelta(minutes=15)
    lamp['forecast_plus_30'] = lamp['forecast_timestamp'] + pd.Timedelta(minutes=30)
    lamp['forecast_plus_45'] = lamp['forecast_timestamp'] + pd.Timedelta(minutes=45)
    # Drop timestamp column
    lamp = lamp.drop('timestamp', axis=1)
    #Filter timestamps in lamp that I want to merge with pushback
    lamp1 = lamp[lamp['timestamp_plus_30'] == lamp['forecast_timestamp']]
    lamp2 = lamp[lamp['timestamp_plus_45'] == lamp['forecast_plus_15']]
    lamp3 = lamp[lamp['timestamp_plus_60'] == lamp['forecast_plus_30']]
    lamp4 = lamp[lamp['timestamp_plus_75'] == lamp['forecast_plus_45']]

    # merge the dataframes based on multiple conditions
    # Inner Merge, note that this inner merge exlucdes all values in pushback where there is no lamp data 
    merge1 = pd.merge(pushback, lamp1, how='inner', left_on=['timestamp'], right_on=['timestamp_plus_30'])
    merge2 = pd.merge(pushback, lamp2, how='inner', left_on=['timestamp'], right_on=['timestamp_plus_45'])
    merge3 = pd.merge(pushback, lamp3, how='inner', left_on=['timestamp'], right_on=['timestamp_plus_60'])
    merge4 = pd.merge(pushback, lamp4, how='inner', left_on=['timestamp'], right_on=['timestamp_plus_75'])
    #Concat 
    merged_df = pd.concat([merge1, merge2, merge3, merge4], ignore_index=True)
    #Get the rows from pushback with no lamp data (lamp data is filled with nan)
        #Take the columns from the merged_df that I want to merge with pushback
    temp1 = merged_df[['gufi', 'timestamp', 'minutes_until_pushback']]
        #Outer merge pushback with temp1 so I can see which rows are in pushback but not in merged_df
    temp2 = pd.merge(pushback, temp1, how='outer', on=['gufi', 'timestamp', 'minutes_until_pushback'], indicator=True)
        #Get only the rows where lamp data is not available
    result_df = temp2[temp2['_merge'] == 'left_only'].drop('_merge', axis=1)
        #Now I have all original rows from the original pushback, and if lamp data is not available, those are filled with nan
    pushback = pd.merge(merged_df, result_df, on=['gufi', 'timestamp', 'airport', 'minutes_until_pushback'], how='outer', indicator=True).drop('_merge', axis=1)
        #Remove columns I added for merging purposes
    pushback = pushback[['gufi', 'timestamp', 'airport', 'minutes_until_pushback', 'forecast_timestamp', 'temperature', 'wind_speed', 'wind_gust', 'cloud_ceiling', 'visibility', 'cloud', 'lightning_prob', 'precip']]


    #Merge mfs onto pushback
        #drop unnecessary column    
    mfs = mfs.drop('isdeparture', axis=1)
        #Merge
    pushback = pd.merge(pushback, mfs, how='left', left_on=['gufi'], right_on=['gufi'])

    #Merges etd ont pushback
    #Rounds timestamp to nearest 15 minutes
    etd['rounded_timestamp'] = etd['timestamp'].dt.round('15min')
    #I want all values to round up so if rounded_timestamp is less than timestamp, it adds 15 minutes
    etd.loc[etd['timestamp'] > etd['rounded_timestamp'], 'rounded_timestamp'] += pd.Timedelta(minutes=15)
    # Drop Duplicates and keep more recent predicted time 
        # Sort etd by timestamp in descending order (note that drop_duplicates keeps first occurence)
    etd = etd.sort_values('timestamp', ascending=False)
        # Drop duplicates
    etd.drop_duplicates(inplace=True, subset=['gufi', 'rounded_timestamp'])
    #Drop timestamp column 
    etd = etd.drop('timestamp', axis=1)
    etd = etd.reset_index(drop=True)
    #Left Merge pushback and etd on gufi
    pushback = pd.merge(pushback, etd, on='gufi', how='left')
    #Take out all observations that rounded_timestamp occurs after timestamp
    pushback = pushback[pushback['timestamp'] >= pushback['rounded_timestamp']]
    #Sort values and drop duplicates
    pushback = pushback.sort_values('rounded_timestamp', ascending=False)
    pushback.drop_duplicates(inplace=True, subset=['gufi', 'timestamp', 'minutes_until_pushback'])
    #Drop rounded_timestamp
    pushback = pushback.drop('rounded_timestamp', axis=1)
    pushback = pushback.sort_values(['gufi', 'timestamp'])
    pushback = pushback.reset_index(drop=True)

    # Pushback is typically about 15 minutes before departure so
        #Estimate the time pushback occurs
    pushback['benchmark_pushback_estimated_time'] = pushback['departure_runway_estimated_time'] - pd.Timedelta(minutes=15)
        #Create a benchmark time
    pushback['benchmark_pushback'] = (pushback['benchmark_pushback_estimated_time'] - pushback['timestamp']) / pd.Timedelta(minutes=1)

    # Create new variables for day of week and month and year
    pushback['day_of_week'] = pushback['departure_runway_estimated_time'].dt.day_name()
    pushback['month'] = pushback['departure_runway_estimated_time'].dt.strftime('%B')
    pushback['hour'] = pushback['departure_runway_estimated_time'].dt.hour
    pushback['year'] = pushback['departure_runway_estimated_time'].dt.year

    #Fill categorical NA 
    pushback['aircraft_type'].fillna('OTHER', inplace=True)
    pushback['flight_type'].fillna('OTHER', inplace=True)
    pushback['major_carrier'].fillna('OTHER', inplace=True)

    #drop uneccessary variables
    drop_cols = ['timestamp', 'airport', 'forecast_timestamp', 'departure_runway_estimated_time', 'benchmark_pushback_estimated_time']
    pushback = pushback.drop(drop_cols, axis=1)
    pushback = pushback.reset_index(drop=True)

    return pushback
    

In [3]:
def onehot (pushback):
    #Drop Airport
    pushback = pushback.drop('airport', axis=1)
    
    #One-hot encoding
    categorical_cols = ['cloud', 'lightning_prob', 'precip', 'day_of_week', 'month', 'aircraft_engine_class', 'flight_type', 'aircraft_type', 'major_carrier', 'year']

    pushback_onehot = pushback
    for col in categorical_cols:
        pushback_onehot[col] = pushback_onehot[col].astype('category')

    return pushback_onehot

In [4]:
def load_model(solution_directory: Path, airport_code: str) -> Any:
    """Load a specific model asset from disk."""
    model_filename = f"models/{airport_code}_model.txt"
    model = lgb.Booster(model_file=str(solution_directory / model_filename))
        
    return model


In [5]:
# Load Models
solution_directory = Path(".")  # Set the solution directory to the current directory

airport_codes = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA', 'KATL_na', 'KCLT_na', 'KDEN_na', 'KDFW_na', 'KJFK_na', 'KMEM_na', 'KMIA_na', 'KORD_na', 'KPHX_na', 'KSEA_na']

models = {}
for airport_code in airport_codes:
    models[airport_code] = load_model(solution_directory, airport_code)


In [6]:
#Append airport test data for all airports

#Create empty dataframe to append airport test data
combined_pushback = pd.DataFrame()

airports = ['KATL', 'KCLT', 'KDEN', 'KDFW', 'KJFK', 'KMEM', 'KMIA', 'KORD', 'KPHX', 'KSEA']

for i in airports:
    #Read in data for train data
    airport = i

    submission_format = f'data/submission_format.csv'
    #submission_format = f'data/code_execution_development_data/submission_format.csv'
    pushback_predict = pd.read_csv(submission_format, parse_dates=['timestamp'])
    pushback_predict = pushback_predict[pushback_predict['airport'] == f"{airport}"]

    etd_label = f'data/{airport}/{airport}_etd.csv.bz2'
    #etd_label = f'data/code_execution_development_data/{airport}/{airport}_etd.csv.bz2'
    etd = pd.read_csv(etd_label, compression='bz2', parse_dates=['timestamp', 'departure_runway_estimated_time'])

    lamp_label = f'data/{airport}/{airport}_lamp.csv.bz2'
    #lamp_label = f'data/code_execution_development_data/{airport}/{airport}_lamp.csv.bz2'
    lamp = pd.read_csv(lamp_label, compression='bz2', parse_dates=['timestamp', 'forecast_timestamp'])

    mfs_label = f'data/{airport}/{airport}_mfs.csv.bz2'
    #mfs_label = f'data/code_execution_development_data/{airport}/{airport}_mfs.csv.bz2'
    mfs = pd.read_csv(mfs_label, compression='bz2')

    #Clean Data
    pushback_predict = clean_data(pushback_predict, etd, lamp, mfs)
    pushback_predict['airport'] = airport
    print(airport)
    print(pushback_predict['gufi'].head(1))
    combined_pushback = pd.concat([combined_pushback, pushback_predict])

KATL
0    AAL1008.ATL.DFW.210607.2033.0110.TFM
Name: gufi, dtype: object
KCLT
0    AAL1005.CLT.LAS.210826.1259.0084.TFM
Name: gufi, dtype: object
KDEN
0    AAL1046.DEN.JAC.210205.2011.0001.TFM_TFDM
Name: gufi, dtype: object


/var/folders/_g/vxz07v2s2pdb7dc0707cwt1c0000gn/T/ipykernel_74716/647321355.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  mfs = pd.read_csv(mfs_label, compression='bz2')


KDFW
0    AAL1001.DFW.CUN.210201.1626.0083.TFM
Name: gufi, dtype: object
KJFK
0    AAL1.JFK.LAX.210202.1357.0089.TFM
Name: gufi, dtype: object
KMEM
0    AAL1148.MEM.DFW.210606.0128.0003.TFM_TFDM
Name: gufi, dtype: object
KMIA
0    AAL1007.MIA.POP.210210.1746.0072.TFM
Name: gufi, dtype: object
KORD
0    AAL1002.ORD.DCA.210201.1457.0077.TFM
Name: gufi, dtype: object
KPHX
0    AAL1078.PHX.LAX.210826.1304.0054.TFM
Name: gufi, dtype: object


/var/folders/_g/vxz07v2s2pdb7dc0707cwt1c0000gn/T/ipykernel_74716/647321355.py:27: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  mfs = pd.read_csv(mfs_label, compression='bz2')


KSEA
0    AAL1006.SEA.DFW.210827.1300.0052.TFM_TFDM
Name: gufi, dtype: object


In [7]:
# Create an empty dictionary to store the subsets of data
airport_data = {}

# Loop over the airports and create subsets of data for each airport
for airport in airports:
    # Create a subset of data for the current airport
    airport_subset = combined_pushback[combined_pushback['airport'] == airport]

    # Apply one-hot encoding to subset
    airport_test = onehot(airport_subset)

    #Create response and predictor variables
    X = airport_test.drop('minutes_until_pushback', axis=1)
    X = X.drop('gufi', axis=1)
    y = airport_test['minutes_until_pushback']

    #Drop columns with Nan data
    X_na = X.dropna(axis=1)

    #Make prediction for normal data
    y_pred = models[airport].predict(X).round().astype(int)

    #Make prediction for data with weather NaNs
    y_pred_na = models[airport + "_na"].predict(X_na).round().astype(int)

    #Change predictions to NaN predictions if weather is Nan
    nan_rows = X.isna().any(axis=1).to_numpy()
    for i, has_nan in enumerate(nan_rows):
        if has_nan:
            y_pred[i] = y_pred_na[i]
    
    # Store the resulting one-hot encoded DataFrame in the airport_data dictionary
    airport_data[airport] = y_pred
    

In [8]:
# Create an empty list to store the predictions from all airports
all_predictions = []

# Loop over the airports and append the predictions to the all_predictions list
for airport in airports:
    all_predictions.extend(airport_data[airport])

# Convert the all_predictions list to a NumPy array
all_predictions_array = np.array(all_predictions)

In [9]:
submission = pd.read_csv(submission_format, parse_dates=['timestamp'])

results_format = f'data/code_execution_development_data/test_labels.csv'
results = pd.read_csv(results_format, parse_dates=['timestamp'])

In [10]:
submission['minutes_until_pushback'] = all_predictions_array

In [21]:
submission.to_csv('open_area_submission.csv', index=False)